In [1]:
##data load ### cpu 버전

import os
import pickle
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
from data_cpu import Dataset, LMOrderedIterator
from model import TFTransfoXLModel,TFTransfoXLLMHeadModel

from transformers import TransfoXLConfig


config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/wiki_short/',
    dataset = 'wt103',
    d_embed=128,
    d_head = 32,
    d_model=128,
    mem_len=400,
    n_head=8,
    n_layer=6,
    batch_size = 18,
    tgt_len = 36,
    ext_len = 0,
    eval_tgt_len = 36
)



kwargs = {}
if config_xl.dataset in ['wt103', 'wt2']:
    kwargs['special'] = ['<eos>','<UNK>']
    kwargs['lower_case'] = False

dataset = Dataset(**kwargs)

train_dataset, val_dataset, test_dataset = dataset.make_dataset(config_xl.data,config_xl.dataset)

# strategy = tf.distribute.MirroredStrategy()

2023-11-10 13:15:58.970463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 13:15:59.516707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-10 13:15:59.516750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-10 13:15:59.516756: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Producing dataset wt103...
final vocab size 109697 from 109695 unique tokens


분산처리를 하기위해서는 train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
이 코드를 실행해야하는데ㅐ 기본 텐서 구조는 안되더라 그래서 텐서 슬라이스 변경해서 이 구조로 바꾸고 데이터를 이터레이션 해서 모델에 집어 넣어야 하는 방식을 고민중 


In [2]:
def gen(data,bsz,bptt,ext_len=None,):
  
  bsz = bsz#3 #60
  bptt = bptt#36 #70
  ext_len = ext_len if ext_len is not None else 0
  data = data
  
  
  # Work out how cleanly we can divide the dataset into bsz parts.
  # 아래의 두 코드는   data 텐서에서 배치 크기 bsz로 깔끔하게 맞지 않는 추가 요소를 제거하는 것 배치에 띡 떨어지게
  n_step = len(data) // (bsz*bptt)
  print('n_step',n_step) # 312779
  
  sliced_data = tf.slice(data,[0],[n_step * bsz*bptt])  
  # print('sliced_data',len(sliced_data))
  # sliced_data = self.data[:self.n_step * self.bsz]
  '''# 시작 위치와 슬라이싱할 크기 설정
  begin = [0]  # 첫 번째 차원의 시작 위치는 0
  size = [6]   # 첫 번째 차원에서 6개의 원소를 슬라이싱

  # 데이터를 잘라내기 (tf.slice 사용)
  sliced_data = tf.slice(data, begin, size)  '''

  # Evenly divide the da
  # ta across the bsz batches.


  new_shape = (bsz, -1)  # 나머지 차원은 자동으로 계산됨
  data_reshaped = tf.reshape(sliced_data, new_shape)
  # data_transposed = tf.transpose(data_reshaped)
  data = data_reshaped
  # print('data',len(data))
  split_num = 2 #GPU num


  # first_half, second_half = tf.split(data, num_or_size_splits=split_num, axis=1)

  n_batch = (n_step + bptt - 1) // bptt

  for i in range(0, len(data[1]) - 1, bptt):
    
    seq_len = min(bptt, data.shape[1] - 1 - i) # # i값이 103227020를 넘지 않는 이상 seq_len = 70


    end_idx = i + seq_len # 70,71,72,73,74......
    beg_idx = max(0, i - ext_len) # 0,1,2,3,4,5
    ''' 아래 처럼 첫번째 차원을 자르는 이류
    로,또,1,등,당,첨 = > 로,또,1    => 로, 등
                    등,당,첨         또, 당
                                    1, 첨
    '''

    p_data = data[:,beg_idx:end_idx] # self.data[:,0:70],[:,1:71] ~
    target = data[:,i+1:i+1+seq_len]

    # second_half_data = second_half[:,beg_idx:end_idx] # self.data[:,0:70],[:,1:71] ~
    # second_half_target = second_half[:,i+1:i+1+seq_len]
    if i + bptt < len(data[1]) - 1:
      yield p_data, target
      # yield second_half_data, second_half_target


In [3]:
dataset = tf.data.Dataset.from_generator(
     gen,
     output_signature=(
         tf.TensorSpec(shape=None, dtype=tf.int32),
         tf.TensorSpec(shape=None, dtype=tf.int32),
         ),
     args=(train_dataset,config_xl.batch_size,config_xl.tgt_len)
         )

2023-11-10 13:16:05.084625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-10 13:16:05.085006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-10 13:16:05.090363: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-10 13:16:05.090722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-10 13:16:05.091061: I tensorflow/compiler/xla/stream_executo

In [4]:
count =0 
for sample in dataset:
    count += 1
print(count)

n_step 8688
8687


In [5]:
first_half_dataset = dataset.take(count//2)
second_half_dataset = dataset.skip((count // 2) + (count % 2))


In [6]:
# counts =0 
# for sample in second_half_dataset:
#     counts += 1
# print(counts)

In [7]:
# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# BATCH_SIZE_PER_REPLICA = 1
# GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 2

In [8]:
# train_dist_dataset = strategy.experimental_distribute_dataset(dataset)


In [9]:
# batched_dataset =dataset.batch(GLOBAL_BATCH_SIZE)
# batched_dataset.cache().prefetch(tf.data.AUTOTUNE)

In [10]:
# count =0 
# for sample in batched_dataset:
#     print(sample[0].shape)
#     print(sample[1].shape)
#     count += 1
# print(count)

In [11]:
# import horovod.tensorflow as hvd


# # Horovod 초기화
# hvd.init()

In [12]:
# class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
#     def __init__(self, d_model, warmup_steps=4000):
#         super(CustomSchedule, self).__init__()

#         self.d_model = tf.cast(d_model, tf.float32)
#         self.warmup_steps = tf.cast(warmup_steps, tf.float32)

#     def __call__(self, step):
#         step = tf.cast(step, tf.float32)
#         arg1 = tf.math.rsqrt(step)
#         arg2 = step * (self.warmup_steps ** -1.5)
        
#         # Horovod의 크기(hvd.size())로 학습률을 스케일링합니다.
#         return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2) * hvd.size()

# @tf.keras.utils.register_keras_serializable()
# class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
#     def __init__(self, d_model, warmup_steps=4000):
#         super(CustomSchedule, self).__init__()

#         self.d_model = tf.cast(d_model, tf.float32)
#         self.warmup_steps = tf.cast(warmup_steps, tf.float32)

#     def __call__(self, step):
#         step = tf.cast(step, tf.float32)  # step을 float32로 명시적 캐스팅
#         arg1 = tf.math.rsqrt(step)
#         arg2 = step * (self.warmup_steps ** -1.5)
        
#         # tf.math.rsqrt(self.d_model)를 float32로 캐스팅합니다.
#         # hvd.size()는 이미 float32로 캐스팅되었다고 가정합니다.
#         return tf.math.rsqrt(self.d_model, tf.float32) * tf.math.minimum(arg1, arg2) * tf.cast(hvd.size(), tf.float32)


    
#     def get_config(self):
#         return {
#             'd_model': self.d_model if isinstance(self.d_model, float) else self.d_model.numpy(),  
#             'warmup_steps': self.warmup_steps if isinstance(self.warmup_steps, float) else self.warmup_steps.numpy()
#         }

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = tf.cast(d_model, tf.float32)

    self.warmup_steps = tf.cast(warmup_steps,tf.float32)

  def __call__(self, step):
    step =tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [13]:
learning_rate = CustomSchedule(config_xl.d_model)


In [14]:
# with strategy.scope():
def compute_loss(loss):

    return tf.nn.compute_average_loss(loss, global_batch_size=2)

In [15]:


# optimizer1 = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
#                                      epsilon=1e-9)
# optimizer2 = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
#                                      epsilon=1e-9)

In [16]:

model0 = TFTransfoXLLMHeadModel(config=config_xl)


model1 = TFTransfoXLLMHeadModel(config=config_xl)

In [18]:

def train_step(model,data,target,mems,optimizer):


  with tf.GradientTape() as tape:
    outputs = model(input_ids=data,labels=target,mems=mems)
    # loss = compute_loss(outputs.loss)
    mems = outputs.mems

  gradients = tape.gradient(outputs.loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return mems,outputs.loss

In [20]:


for epoch in range(EPOCHS):
    start = time.time()
    mems1 = None
    mems2 = None
    total_loss = 0.0
    num_batches = 0
    for (frist_data, frist_target), (second_data,second_target) in zip(first_half_dataset,second_half_dataset):
        with tf.device('/GPU:0'):

            mems,loss = train_step(model0,frist_data,frist_target,mems1,optimizer1)
            print('loss',loss)
            total_loss += loss[0].numpy()
            if num_batches % 50 == 0:
                print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss}')
        # 가중치 복사
        weights_gpu0 = model0.get_weights()
        
        with tf.device('/GPU:1'):

            mems,loss = train_step(model1,second_data,second_target,mems2,optimizer2)
            total_loss += loss[0].numpy()
            num_batches += 1
            if num_batches % 50 == 0:
                print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss}')



        # 가중치 복사
        weights_gpu1 = model1.get_weights()
        
        # 가중치 평균 계산 및 동기화
        average_weights = [(w0 + w1) / 2.0 for w0, w1 in zip(weights_gpu0, weights_gpu1)]
        
        # 두 모델의 가중치 업데이트
        model0.set_weights(average_weights)
        model1.set_weights(average_weights)

# 에포크마다 평균 손실을 기록
    avg_loss = total_loss / num_batches
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}, Time: {time.time() - start}')


        #     if num_batches % 50 == 0:
        #         print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss}')
            
        # train_loss = total_loss / num_batches
            
            
        # template = ("Epoch {}, Loss: {} ")
        # print(template.format(epoch + 1, train_loss,
        #                         )) 
# checkpoint_path = "./checkpoints/train"

    # ckpt = tf.train.Checkpoint(transformer=model,
    #                         optimizer=optimizer)

    # ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

    # # if a checkpoint exists, restore the latest checkpoint.
    # if ckpt_manager.latest_checkpoint:
    #     ckpt.restore(ckpt_manager.latest_checkpoint)
    #     print('Latest checkpoint restored!!')

n_step 8688
n_step 8688


2023-11-10 13:16:48.185113: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x563787750180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-10 13:16:48.185133: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-11-10 13:16:48.185136: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-11-10 13:16:48.188500: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-10 13:16:48.266089: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


loss [<tf.Tensor: shape=(), dtype=float32, numpy=10.723874>]
Epoch 1 Batch 0 Loss [<tf.Tensor: shape=(), dtype=float32, numpy=10.723874>]


AttributeError: 'list' object has no attribute 'dtype'